# Solow Model

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code from [lecture 5](https://numeconcopenhagen.netlify.com/lectures/Workflow_and_debugging).
> 1. Remember this [guide](https://www.markdownguide.org/basic-syntax/) on markdown and (a bit of) latex.
> 1. Turn on automatic numbering by clicking on the small icon on top of the table of contents in the left sidebar.
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [1]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

# Model description


We consider the **standard Solow-model** where:

* $K_t$ is capital
* $L_t$ is labor (growing with a constant rate of $n$)
* $A_t$ is technology (growing with a constant rate of $g$)
* $Y_t = F(K_t,A_tL_t)$ is GDP

**Saving** is a constant fraction of GDP

$$ S_t = sY_t,\,\qquad s\in(0,1) $$

such that **capital accumulates** according to

$$ K_{t+1}=S_{t}+(1-\delta)K_{t}=sF(K_{t},A_{t}L_{t})+(1-\delta)K_{t}, \qquad \delta \in (0,1) $$

The parameter $\delta$ is the depreciation rate.

The technology $A_t$ is evolving according to

$$A_{t+1}=(1+g)A_t$$

where $g$ is the exogenous growth rate.

The labor $L_t$ is evolving according to

$$L_{t+1}=(1+n)L_t$$

where $n$ is the exogenous growth rate.

The **production function** has **constant-return to scale** such that

$$ \tilde{y_t}= \frac{Y_{t}}{A_{t}L_{t}}=\frac{F(K_{t},A_{t}L_{t})}{A_{t}L_{t}}=F(\tilde{k}_{t},1)\equiv f(\tilde{k}_{t}) $$

where $\tilde{k}_t = \frac{K_t}{A_{t}L_{t}}$ is the technology adjusted capital-labor ratio.

The **transition equation** then becomes

$$ \tilde{k}_{t+1}= \frac{1}{(1+n)(1+g)}[sf(\tilde{k}_{t})+(1-\delta)\tilde{k}_{t}] $$

We assume the **production function** is **Cobb-Douglas** so

$$ F(K_{t},A_{t}L_{t})=K_{t}^{\alpha}(A_{t}L_{t})^{1-\alpha}\Rightarrow f(\tilde{k}_{t})=\tilde{k}_{t}^{\alpha} $$

# Steady state

## Analytical solution

We can solve the transition equation by defining the steady state equation. We set $\tilde{k}_{t+1} = \tilde{k}_{t} = \tilde{k}^*$ and solve it algebraically by solving for $\tilde{k}^*$.

We can use **sympy** to find an analytical expression for the steady state, i.e. solve

$$ \tilde{k}^{\ast}= \frac{1}{(1+n)(1+g)}[sf(\tilde{k}^{\ast})+(1-\delta)\tilde{k}^{\ast}] $$

First we define all **symbols**:

In [2]:
#initialize variables in Sympy
k = sm.symbols('k')
alpha = sm.symbols('alpha')
delta = sm.symbols('delta')
s = sm.symbols('s')
g = sm.symbols('g')
n = sm.symbols('n')

Then we define the **steady state equation**

In [3]:
ss = sm.Eq(k,(s*k**alpha+(1-delta)*k)/((1+n)*(1+g)))

and **solve** it

In [4]:
kss = sm.solve(ss,k)[0]
kss

((delta + g*n + g + n)/s)**(1/(alpha - 1))

For later use, we turn the solution into a **Python function**

In [5]:
ss_func = sm.lambdify((s,g,n,alpha,delta),kss)

## Numerical solution

We can re-write the equation for the steady state capital per capita as

\\[ 0 = \frac{1}{(1+n)(1+g)}[sf(\tilde{k}^{\ast})+(1-\delta)\tilde{k}^{\ast}] - \tilde{k}^{\ast} \\]

whereby it clearly becomes a **root-finding problem**. Such a problem can be solved by a **bisection method**.

Define the model **parameters**:

In [6]:
s = 0.2
g = 0.01
n = 0.01
alpha = 1/3
delta = 0.1

**Solve numerically** for the steady state:

In [7]:
solution = modelproject.solve_for_ss(s,g,n,alpha,delta)

print(f'analytical solution is: {ss_func(s,g,n,alpha,delta):.3f}')
print(f' numerical solution is: {solution.root:.3f}')

analytical solution is: 2.149
 numerical solution is: 2.149


# Further analysis

ADD FURTHER ANALYSIS, VISUALIZATIONS AND EXTENSIONS.

We want to investigate the capital-output ratio. This is definded as:

$$z_t=\frac{\frac{K_t}{A_tL_t}}{\frac{Y_t}{A_tL_t}}$$

Therefore we have the definition:

$$z_t \equiv \frac{\tilde{k_t}}{\tilde{y_t}}$$

The transition equation then becomes:

$$z_{t+1}=\bigg(\frac{1}{(1+n)(1+g)}\bigg)^{1-\alpha}z_t^{\alpha}(s+(1-\delta)z_t)^{1-\alpha}$$

We set $z_{t+1}=z_t=z^*$ and this is the steady state equation:

$$z^{\ast}=\bigg(\frac{1}{(1+n)(1+g)}\bigg)^{1-\alpha}z^{\ast \alpha}(s+(1-\delta)z^{\ast})^{1-\alpha}$$

## Analytical solution

We define the new symbol in Symby

In [14]:
#initialize variables in Sympy
z = sm.symbols('z')
k = sm.symbols('k')
alpha = sm.symbols('alpha')
delta = sm.symbols('delta')
s = sm.symbols('s')
g = sm.symbols('g')
n = sm.symbols('n')

Then we define the new steady state equation

In [9]:
ss_new = sm.Eq(z,(s+(1-delta)*z)/((1+n)*(1+g)))


and solve it

In [10]:
zss = sm.solve(ss_new,z)[0]
zss

s/(delta + g*n + g + n)

We save the solution as a **Python function**

In [11]:
zss_func = sm.lambdify((s,g,n,delta),zss)

## Numerical solution

We define the model **parameters** and solve the model numerically

In [23]:
#Set parameter values
s = 0.2
g = 0.01
n = 0.01
alpha = 1/3
delta = 0.1
    
    
# Define objective function
f = lambda z: z**alpha
obj_zss = lambda zss: zss - f(zss)*(s+(1-delta)*zss)/((1+n)(1+g))**(1-alpha)
                             
# Call root finder, and optimizing using the bisect method
result_new = optimize.root_scalar(obj_zss,bracket=[0.1,100],method='bisect')
    

print(f'analytical solution is: {zss_func(s,g,n,delta):.3f}')
print(f'numerical solution is: {result_new.root:.3f}')

TypeError: 'float' object is not callable

# Conclusion

ADD CONCISE CONCLUSION.